# Apple Appstore Review scrape

In [2]:
import pandas as pd
import sqlalchemy as db

## create db connection

i used a local docker with an mariadb. if you want you can also use the local apple_app_id.json file which is a backup of my sql data

In [3]:

db_engine = "mariadb+pymysql"
address = "127.0.0.1"
port = 3306
username = "root"
password = "itsonlylocal"
database = "mood_tracker"
engine = db.create_engine(f'{db_engine}://{username}:{password}@{address}:{port}/{database}')


load sql data as pandas df

In [8]:
df = pd.read_sql_table('apple_apps_appId', engine)

# df = pd.read_json("../data_backup/apple_apps_appId.json")

df.head()

,id,appstore_id,appName,app_id
0,1,1305375832,Woebot: Your Self-Care Expert,1840
1,2,1194023242,"Daylio Journal - Goals, Habits",2157
2,3,1241229134,Reflectly - Journal & AI Diary,2177
3,4,1439590239,Alan Mind Daily Journal,2459
4,5,1300107718,#SelfLove,2460


define apple rss response, if you have an apple developer it is recommended using their [developer api](https://developer.apple.com/documentation/appstoreconnectapi/app_store/customer_reviews) which is not limited.


In [26]:
import requests
import json
region = 'us'
id = 1194023242 # app id
sort = 'mostHelpful' # other option: mostRecent
pageNr = 1 # max 10
page = requests.get(f"https://itunes.apple.com/{region}/rss/customerreviews/id={id}/sortBy={sort}/page={pageNr}/json")
result = json.loads(page.content)
entry = result["feed"]["entry"][0]
    
entry  


{'author': {'uri': {'label': 'https://itunes.apple.com/us/reviews/id427466927'},
  'name': {'label': 'CatLover1707'},
  'label': ''},
 'updated': {'label': '2022-05-04T07:12:01-07:00'},
 'im:rating': {'label': '5'},
 'im:version': {'label': '1.42.1'},
 'id': {'label': '8636472890'},
 'title': {'label': 'Wonderful Mood Tracker'},
 'content': {'label': 'I was looking for a mood tracking app where I could have multiple entries throughout the day, not just one at the end. I use this app to help track my anxiety and how I feel leading up to an anxious period as well as other moods. I love that I can go back and add entries even days later if I forgot. I also love that it’s completely customizable and I can have tons of options. I did end up upgrading to premium to unlock unlimited moods and activities because I find it really beneficial and absolutely worth the around $20 a year. I have an activities folder with ranges for my heart rate so so can track that during an anxiety attack, I have 

In [53]:
def unpack_entry(entry: dict) -> dict:
    """Unpacks the entry dict into a new dict with only the relevant data"""
    return {
        "reviewId": entry["id"]["label"],
        "author": entry["author"]["name"]["label"],
        "title": entry["title"]["label"],
        "content": entry["content"]["label"],
        "rating": entry["im:rating"]["label"],
        "version": entry["im:version"]["label"],
        "voteSum": entry["im:voteSum"]["label"],
        "voteCount": entry["im:voteCount"]["label"],
        "updated": entry["updated"]["label"],
    }
    
    
unpack_entry(entry)

{'reviewId': '8636472890',
 'author': 'CatLover1707',
 'title': 'Wonderful Mood Tracker',
 'content': 'I was looking for a mood tracking app where I could have multiple entries throughout the day, not just one at the end. I use this app to help track my anxiety and how I feel leading up to an anxious period as well as other moods. I love that I can go back and add entries even days later if I forgot. I also love that it’s completely customizable and I can have tons of options. I did end up upgrading to premium to unlock unlimited moods and activities because I find it really beneficial and absolutely worth the around $20 a year. I have an activities folder with ranges for my heart rate so so can track that during an anxiety attack, I have one with different physical symptoms like sweaty, dizzy, etc., and even a folder for methods and techniques I’ve already tried to calm down. I hate having to type out every little thing so being able to just go through and simply pick what applies at 

In [55]:
def get_all_rss_reviews_entries(appId: int,appName: str, region='us', sort='mostHelpful') -> list:
    """Gets all reviews from the RSS feed"""
    maxPage = 10
    entries = []
    for pageNr in range(1, maxPage + 1):
        result = requests.get(f"https://itunes.apple.com/{region}/rss/customerreviews/id={appId}/sortBy={sort}/page={pageNr}/json")
        if result.status_code != 200:
            print(f"{appName} doesn't have page {pageNr} Error: {result.status_code}")
            break
        result = json.loads(result.content)
        result_entries = map(unpack_entry, result["feed"]["entry"])
        entries.extend(result_entries)
    df = pd.DataFrame(entries)
    df["appName"] = appName
    df["appstore_id"] = appId
    print(f"{appName} has {len(entries)} reviews")
    return df


reviews = pd.DataFrame(columns=["appName","appstore_id", "author", "title", "content", "rating", "version", "voteSum", "voteCount", "updated" "platform", "reviewId"])
pd.concat([reviews, pd.DataFrame(get_all_rss_reviews_entries(1194023242, "Dalio "))]).head(10)


,appName,appstore_id,author,title,content,rating,version,voteSum,voteCount,updatedplatform,reviewId,updated
0,Dalio,1194023242,CatLover1707,Wonderful Mood Tracker,I was looking for a mood tracking app where I ...,5,1.42.1,9,13,NaN,8636472890,2022-05-04T07:12:01-07:00
1,Dalio,1194023242,"Don't Usually Review, but ...","Great insight into yourself, simple, useful - ...",I have always loved to journal and document bo...,5,1.8.2,38,49,NaN,3264361324,2018-10-04T17:10:05-07:00
2,Dalio,1194023242,ej blu,Changed My Life!!! ❤️,I have been using this app for at least a few ...,5,1.25.0,16,20,NaN,6296401649,2020-08-08T15:50:34-07:00
3,Dalio,1194023242,Iceberg Support,An indispensable tool,Dear Dalio Team. Congratulations on building a...,5,1.38.1,0,0,NaN,8152136948,2021-12-20T19:05:47-07:00
4,Dalio,1194023242,luna.xz,Only one complaint (updated no complaint),This app is super helpful for keeping track of...,5,1.29.3,3,3,NaN,6771352520,2020-12-19T10:35:41-07:00
5,Dalio,1194023242,melaward,Amazingly Simple Mood and Life Data Tracker,This is the first review I’ve ever written for...,5,1.10.3,4,5,NaN,3881079412,2019-03-14T10:28:55-07:00
6,Dalio,1194023242,SytherMinatusaki94,Very awesome mood tracker!,I love this app. So much so I even invested in...,5,1.39.1,0,0,NaN,8357076694,2022-02-14T20:54:42-07:00
7,Dalio,1194023242,sheryqiao,Changed me so much,I hate diaries. I can never keep a diary for l...,5,1.34.2,2,2,NaN,7525687505,2021-06-30T22:20:06-07:00
8,Dalio,1194023242,Zoe Lakiya,"Great App for goals, routine, recovery, mood p...",This app has been incredibly helpful for me. I...,5,1.38.1,0,0,NaN,8160577444,2021-12-23T06:18:08-07:00
9,Dalio,1194023242,Hthrglass,Daylio is a Must-lio,Last year I spent a lot of time really trying ...,5,1.30.0,1,2,NaN,6901078222,2021-01-22T11:09:47-07:00


In [56]:
from tqdm import tqdm



for index, row in tqdm(df.iterrows()):
    reviews = pd.concat([reviews, pd.DataFrame(get_all_rss_reviews_entries(row["appstore_id"], row["appName"]))])
    
    
    

    

4it [00:29,  7.38s/it]


KeyError: 'entry'